In [59]:
import pandas as pd
from helpers.columns_names_trans import renamed_jams
import numpy as np

In [60]:
df_files = pd.read_csv('./sb-files-export/sb-files-export.csv')
df_files = df_files.rename(columns=renamed_jams)
df_jam_clean = pd.DataFrame()

C:\Users\inanc\AppData\Local\Temp\ipykernel_32924\3454014524.py:1: DtypeWarning: Columns (8,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_files = pd.read_csv('./sb-files-export/sb-files-export.csv')


In [61]:
print(df_files.columns.tolist())

['id', 'version', 'type', 'source', 'start_time', 'end_time', 'situation_created_time', 'situation_version_time', 'special_case', 'vild_version', 'vild_primary_location', 'vild_secondary_location', 'vild_direction', 'vild_distance_to_primary_location', 'vild_distance_to_secondary_location', 'traffic_jam_length_m', 'general_cause_type', 'cause_description', 'cause_type', 'safety_related_message', 'general_publication_message', 'probability', 'location', 'vild_primary_road_number', 'vild_primary_direction', 'vild_primary_destination', 'vild_primary_name', 'vild_primary_hectometer', 'vild_secondary_road_number', 'vild_secondary_direction', 'vild_secondary_destination', 'vild_secondary_name', 'vild_secondary_hectometer']


I print the columns seperate, because there are to many to view in a table. (Just for getting a better look at what it contains)

In [62]:
df_files.sample(3)

,id,version,type,source,start_time,end_time,situation_created_time,situation_version_time,special_case,vild_version,...,vild_primary_road_number,vild_primary_direction,vild_primary_destination,vild_primary_name,vild_primary_hectometer,vild_secondary_road_number,vild_secondary_direction,vild_secondary_destination,vild_secondary_name,vild_secondary_hectometer
47050,NLRWS_0004713621_1,92,slowTraffic,NLRWS,2025-06-03 16:13:33,NaN,2025-06-03 16:13:33,2025-06-03 17:45:10,NaN,6.11A,...,A2,R,Knooppunt (triangle),De Hogt,158.0,A2,R,Afrit,Batadorp/Randweg N2,155.8
49016,NLRWS_0004721288_1,16,slowTraffic,NLRWS,2025-06-06 17:39:34,NaN,2025-06-06 17:39:34,2025-06-06 17:55:15,NaN,6.11A,...,A2,L,Knooppunt (triangle),De Hogt,166.5,A2,L,Knooppunt (triangle),Leenderheide,169.8
74478,NLRWS_0004829063_1,40,stationaryTraffic,NLRWS,2025-08-25 15:40:30,NaN,2025-08-25 15:40:30,2025-08-25 16:20:12,NaN,6.12A,...,A67,R,Knooppunt (triangle),De Hogt,18.3,A67,R,Afrit,Eersel,12.4


#### Data understanding / dictionary (Traffic Jam Dataset)

A **data dictionary** has been created to document all dataset features.  
It specifies the name, type, units, valid range/values, description, source, and quality notes.  
This ensures clarity, consistency, and transparency for all project members and stakeholders.  

| Feature                   | Type     | Units      | Valid Range / Values                       | Description                                                                 | Source | Quality Notes |
|----------------------------|----------|------------|-------------------------------------------|-----------------------------------------------------------------------------|--------|---------------|
| id                         | string   | –          | Unique ID per event                        | Identifier of the traffic jam record                                        | NDW    | Unique key |
| version                    | int      | –          | ≥1                                         | Internal version number of the event                                        | NDW    | Not relevant for analysis |
| type                       | category | –          | {slowTraffic, stationaryTraffic, …}        | Category of jam (slower flow vs standstill)                                 | NDW    | Reliable |
| source                     | string   | –          | {NLRWS}                                    | Data source                                                                 | NDW    | Constant |
| start_time                 | datetime | UTC        | 2025-01 → 2025-09                          | Start time of the traffic jam                                               | NDW    | Always filled |
| end_time                   | datetime | UTC        | Mostly NaN                                 | End time of the jam                                                         | NDW    | Often missing |
| situation_created_time      | datetime | UTC        | 2025-01 → 2025-09                          | When the jam record was created                                             | NDW    | Consistent |
| situation_version_time      | datetime | UTC        | 2025-01 → 2025-09                          | Last update of the jam                                                      | NDW    | Used as fallback for missing `end_time` |
| special_case                | string   | –          | Mostly NaN                                 | Technical field for exceptions                                              | NDW    | Dropped |
| vild_version                | string   | –          | e.g. 6.11A, 6.12A                          | NDW version reference                                                       | NDW    | Metadata |
| vild_primary_road_number    | string   | –          | {A2, A50, A58, A67, N2, …} (6 unique)      | Main road where the jam occurs                                              | NDW    | Needs to match `road_code` |
| vild_primary_direction      | category | –          | {L, R}                                     | Direction of the jam (left/right carriageway)                               | NDW    | Must be mapped to N/S/E/W |
| vild_primary_name           | string   | –          | 41 unique names (e.g., De Hogt, Batadorp)  | Location of the jam (junction or exit)                                      | NDW    | Needs normalization |
| vild_primary_destination    | string   | –          | {Afrit, Knooppunt, …}                      | Type of primary location (exit, junction)                                   | NDW    | Textual label |
| vild_primary_location       | int      | –          | Numeric ID                                 | NDW location identifier                                                     | NDW    | Consistent |
| vild_primary_hectometer     | float    | km         | e.g. 12.1 – 167.7                          | Road position of the jam (hectometer marker)                                | NDW    | Sometimes missing |
| vild_secondary_name         | string   | –          | e.g. Geldrop, Someren                      | Connected location (to/from)                                                | NDW    | Needs normalization |
| vild_secondary_destination  | string   | –          | e.g. Afrit, Knooppunt                      | Type of secondary location                                                  | NDW    | Consistent |
| vild_secondary_location     | float    | –          | ID, sometimes NaN                          | NDW secondary location ID                                                   | NDW    | Mixed type |
| vild_secondary_road_number  | string   | –          | {A2, A67, A58, …}                          | Secondary road number                                                       | NDW    | Matches primary |
| vild_secondary_direction    | category | –          | {L, R}                                     | Secondary road direction                                                    | NDW    | Same as primary |
| vild_secondary_hectometer   | float    | km         | e.g. 14.2 – 31.1                           | Secondary road position (hectometer marker)                                 | NDW    | Sometimes NaN |
| traffic_jam_length_m        | int      | meters     | ~500 – 7,000+                              | Length of the jam in meters                                                 | NDW    | Outliers possible |
| general_cause_type          | category | –          | {ManagedCause, NonManagedCause}            | General classification of cause                                             | NDW    | Always present |
| cause_description           | string   | –          | Free text                                  | Cause description (optional)                                                | NDW    | Sparse |
| cause_type                  | string   | –          | e.g. accident, weather                     | Detailed cause of jam                                                       | NDW    | Often missing |
| safety_related_message      | string   | –          | Mostly NaN                                 | Safety notes from NDW                                                       | NDW    | Empty |
| general_publication_message | string   | –          | Mostly NaN                                 | Message field for public info                                               | NDW    | Empty |
| probability                 | float    | –          | NaN                                        | Event probability                                                           | NDW    | Not used |
| location                    | string   | –          | NaN                                        | General location text                                                       | NDW    | Empty |


### Columns might needed for merging the two datasets

To combine the **traffic speed & intensity dataset** with the **traffic jam dataset**, several key columns are required:

#### 1. Time dimension
- **Intensity dataset:** `start_measurement_period`  
- **Jam dataset:** `start_time`, `end_time`  
Check whether the measurement period falls **within** the reported jam duration.  

#### 2. Road number
- **Intensity dataset:** `road_code`  
- **Jam dataset:** `vild_primary_road_number`  
Ensure both use the same format (e.g., "A2", "A50").  

#### 3. Driving direction
- **Intensity dataset:** `driving_direction`  
- **Jam dataset:** `vild_primary_direction`  
Match direction (L/R = left/right carriageway).  

#### 4. Location / position
- **Intensity dataset:** `name_measuring_location` or `measurement_location_id`  
- **Jam dataset:** `vild_primary_name`, `vild_primary_hectometer`  
Use these to verify whether a measurement point lies **within the jam’s spatial range** (based on hectometer values and jam length). 

Note: Further we shall select only the required key columns for a better overview of data

In [63]:
selectec_columns_present = [
    # tijd
    "start_time", "end_time", "situation_version_time",
    # weg + richting
    "vild_primary_road_number", "vild_primary_direction",
    # locatie-teksten (voor knooppunt/afrit matching)
    "vild_primary_name", "vild_primary_destination", "vild_primary_location",
    "vild_secondary_name", "vild_secondary_destination", "vild_secondary_location",
    # positie + lengte
    "vild_primary_hectometer", "traffic_jam_length_m",
    # context (optioneel)
    "type", "general_cause_type", "cause_type"
]

df_files = df_files[selectec_columns_present]

In [64]:
df_files.sample(5)

,start_time,end_time,situation_version_time,vild_primary_road_number,vild_primary_direction,vild_primary_name,vild_primary_destination,vild_primary_location,vild_secondary_name,vild_secondary_destination,vild_secondary_location,vild_primary_hectometer,traffic_jam_length_m,type,general_cause_type,cause_type
36713,2025-05-06 15:39:33,NaN,2025-05-06 15:43:03,A2,L,De Hogt,Knooppunt (triangle),7918,Leenderheide,Knooppunt (triangle),7912.0,166.3,2900,slowTraffic,ManagedCause,NaN
61263,2025-07-03 08:08:33,NaN,2025-07-03 08:15:14,A67,L,Leenderheide/Randweg N2,Afrit,21884,Someren,Afrit,10038.0,26.5,3400,slowTraffic,ManagedCause,NaN
29732,2025-04-18 12:23:29,NaN,2025-04-18 12:35:20,N2,R,Meerhoven-Zuid,Afrit,22215,A2: Batadorp/Randweg N2,Afrit,21881.0,160.8,3400,slowTraffic,ManagedCause,NaN
73051,2025-08-21 16:47:30,NaN,2025-08-21 16:49:08,A2,R,De Hogt,Knooppunt (triangle),7918,Batadorp,Knooppunt (triangle),7925.0,159.4,2300,slowTraffic,ManagedCause,NaN
12432,2025-02-25 16:24:30,NaN,2025-02-25 16:32:07,A67,L,Valkenswaard,Afrit,10083,De Hogt,Knooppunt (triangle),10025.0,15.3,3200,slowTraffic,ManagedCause,NaN


### Sample Analysis (Selected Traffic Jam Columns)

This table shows **traffic jam events** with key attributes for time, location, length, and cause.  
Each row represents one reported traffic jam situation.

---

#### Observations

1. **Time dimension**  
   - All events have a `start_time`; some `end_time` values are missing → those jams were still ongoing when recorded.  
   - `situation_version_time` is often used as the latest known update.  
   - Example: The jam on *29 May (A2)* has no `end_time` but was last updated at 10:48.

2. **Road and direction**  
   - Jams occur on **A2** and **A67**, with directions marked as `L` (left carriageway) or `R` (right carriageway).  
   - This matches Dutch highway data conventions.

3. **Location**  
   - Locations are given by **nodes** (Batadorp, Leenderheide, De Hogt) and **types** (Knooppunt, Afrit).  
   - Primary and secondary locations describe the jam segment between two points.  
   - Example: Batadorp ↔ Leenderheide appears multiple times, which suggests it is a common bottleneck.

4. **Hectometer and jam length**  
   - `vild_primary_hectometer` pinpoints the exact road position (e.g., 164.2 km).  
   - `traffic_jam_length_m` indicates the jam size:  
     - Shorter jams: ~1952 meters (A67, Geldrop ↔ Leenderheide).  
     - Longer jams: up to 6700 meters (A67, Eersel ↔ De Hogt).  
   - These values are realistic for Dutch highways during peak traffic or accidents.

5. **Jam type and cause**  
   - Types include **slowTraffic** (reduced flow) and **stationaryTraffic** (standstill).  
   - Causes are split into:  
     - `ManagedCause`: recurring traffic management bottlenecks (likely structural).  
     - `NonManagedCause`: unexpected events (accidents).  
   - Example: The 1 May jam on A67 (Eersel ↔ De Hogt) is labeled *stationaryTraffic* with cause *accident*.


#### Interpretation
- The data is **consistent with real-world traffic**: jams are longer during accidents and often located at **knooppunten** (junctions) and **afritten** (exits).  
- Missing `end_time` values mean the dataset captures **live or ongoing events**, not just completed ones.  
- Locations (Batadorp, Leenderheide, De Hogt) are major junctions, confirming that the dataset correctly highlights known bottlenecks.  
- Jam length and cause categories provide useful target labels for linking with speed/intensity data.


In [65]:
df_files.info()
print(df_files.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85645 entries, 0 to 85644
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   start_time                  85645 non-null  object 
 1   end_time                    9310 non-null   object 
 2   situation_version_time      85645 non-null  object 
 3   vild_primary_road_number    85645 non-null  object 
 4   vild_primary_direction      85645 non-null  object 
 5   vild_primary_name           85645 non-null  object 
 6   vild_primary_destination    85645 non-null  object 
 7   vild_primary_location       85645 non-null  int64  
 8   vild_secondary_name         85622 non-null  object 
 9   vild_secondary_destination  85622 non-null  object 
 10  vild_secondary_location     85622 non-null  float64
 11  vild_primary_hectometer     85622 non-null  float64
 12  traffic_jam_length_m        85645 non-null  int64  
 13  type                        856

### Table Analysis (`df_files.info()`)

The dataset contains **85,645 rows** and **33 columns**, describing traffic jam events within the same time period as the traffic speed & intensity dataset.  

- **Data types:**  
  - Most columns are of type `object` → representing **categorical or textual information** (e.g., type of cause, location descriptions, directions).  
  - A smaller number of columns are numeric (`int64`, `float64`) → representing **quantitative values** such as jam length (in meters), distances to reference points, or hectometer coordinates.  

- **Completeness:**  
  - Some columns are fully populated (e.g., `id`, `version`, `type`, `source`).  
  - Others contain **many missing values** (e.g., `end_time`, `special_case`, `cause_description`, `safety_related_message`).  
  - This suggests that not all events are fully documented, or some attributes only apply to specific cases.  

- **Interpretation:**  
  - The dataset is **event-based**: each row represents a reported traffic jam situation.  
  - Columns describe **when** the jam was recorded (`start_time`, `end_time`, `situation_created_time`),  
    **where** it occurred (`primary/secondary road, location, hectometer`), and  
    **why** it occurred (`general_cause_type`, `cause_type`, `cause_description`).  
  - The key numeric indicator is `traffic_jam_length_m`, which specifies the jam length in meters.  


In [66]:
print(df_files.describe(include="all"))

                 start_time             end_time situation_version_time  \
count                 85645                 9310                  85645   
unique                 9810                 8838                  80379   
top     2025-05-20 09:43:33  2025-04-18 15:27:31    2025-04-18 12:17:29   
freq                    418                    3                      4   
mean                    NaN                  NaN                    NaN   
std                     NaN                  NaN                    NaN   
min                     NaN                  NaN                    NaN   
25%                     NaN                  NaN                    NaN   
50%                     NaN                  NaN                    NaN   
75%                     NaN                  NaN                    NaN   
max                     NaN                  NaN                    NaN   

       vild_primary_road_number vild_primary_direction vild_primary_name  \
count                  

### Notes – Traffic Jam Dataset (sb-files-export)

- `end_time` is **missing in most rows** (only ~11% filled).  
  → Use `situation_version_time` as fallback to have a proper end timestamp.

- `cause_type` is **almost always NaN** (only ~13% filled).  
  → Replace with `"unknown"` so we don’t lose records.

- `vild_primary_road_number` has 6 different roads, but **A2 dominates** (over 60% of all jams).  
  → Needs to be standardized to match the `road_code` column in the intensity dataset.

- `vild_primary_direction` only has **L/R**, while intensity dataset uses `northBound/southBound/eastBound/westBound`.  
  → We’ll need a mapping (e.g. L = one carriageway, R = the other) depending on the road.

- Location names (`vild_primary_name`, `vild_secondary_name`) are written in different ways:  
  → Example: “Afrit Geldrop” vs “Geldrop”.  
  → Normalize (lowercase, remove words like “afrit” or “knooppunt”) so they match better with measurement locations.

- `traffic_jam_length_m` values look realistic (2–6 km jams) but we should check for outliers (0 or extreme >10 km).

- Extra columns like `special_case`, `id`, `version` don’t add value for analysis → can be dropped.

## **Preparation Notes**
1. Fill `end_time` with `situation_version_time`.  
2. Replace missing `cause_type` with `"unknown"`.  
3. Standardize road numbers and directions to match the intensity dataset.  
4. Clean up location names for better matching.  
5. Check for outliers in `traffic_jam_length_m`.  
6. Drop unneeded columns.



---

## **Data Preparation**

### **Fill end_time with situation_version_time** 
There are missing end_time values in the traffic jam dataset, there for where end_time is missing we will fill it with situation_version_time. 
because it contains the last known update time for the jam event, which is a reasonable proxy for when the jam might have ended or was last confirmed to be ongoing.

In [67]:

df_files["start_time"] = pd.to_datetime(
    df_files["start_time"], errors="coerce")

df_files["end_time"] = pd.to_datetime(df_files["end_time"], errors="coerce")

df_files["situation_version_time"] = pd.to_datetime(
    df_files["situation_version_time"], errors="coerce")

df_files["situation_version_time"] = pd.to_datetime(
    df_files["situation_version_time"], errors="coerce")

### **Replace missing cause_type with unknown**
cause_type often has missing values, which can lead to loss of information during analysis. To address this, we will replace missing values in the cause_type column with the string "unknown".

In [68]:
df_files["cause_type"] = df_files["cause_type"].replace(
    {None: "unknown", np.nan: "unknown"})

### **Standardize road numbers and directions**
Standardize road numbers and directions to match the intensity dataset.
 This will involve reformatting the road numbers to ensure consistency in how they are represented (e.g., "A1" vs "A-1") and ensuring that direction indicators (e.g., "North", "South") are standardized across both datasets.

In [69]:
df_files['vild_primary_road_number'] = df_files['vild_primary_road_number'].str.replace(
    r'[^A-Za-z0-9]', '', regex=True).str.upper()

In the traffic jam dataset, directions are stored as **L/R** (left/right carriageway).  
In the intensity dataset, directions are expressed as **northBound, southBound, eastBound, westBound**.  

To merge both datasets, we need a consistent mapping.  
The orientation depends on the road:

- **A2 / N2 / A50** → North–South roads  
  - L = southBound, R = northBound  
- **A67 / A58** → East–West roads  
  - L = westBound, R = eastBound  

This ensures that both datasets use the same direction labels, which is required for correct merging and analysis.

**Sources:**  
- NDW Documentation (Nationale Databank Wegverkeersgegevens) → [ndw.nu/datalevering](https://ndw.nu/datalevering/)  
- Rijkswaterstaat roads overview (orientation of Dutch highways) → [rijkswaterstaat.nl/wegen/wegenoverzicht](https://www.rijkswaterstaat.nl/wegen/wegenoverzicht)  

In [70]:
from helpers.Merge_join_helpers import map_direction, normalize_ref_text, to_meaningful_keys


direction_mapping = {
    "A2": {"L": "southBound", "R": "northBound"},    # A2 runs north <-> south
    "A50": {"L": "southBound", "R": "northBound"},   # A50 also mainly N <-> S
    "A58": {"L": "westBound", "R": "eastBound"},     # A58 runs west <-> east
    "A67": {"L": "westBound", "R": "eastBound"},     # A67 runs west <-> east
    "N2": {"L": "southBound", "R": "northBound"}     # N2 is parallel to A2
}

df_files["driving_direction"] = df_files.apply(map_direction, axis=1)
df_files["road_code"] = df_files["vild_primary_road_number"]

In [71]:
df_files.sample(2)  # check direction_standardized

,start_time,end_time,situation_version_time,vild_primary_road_number,vild_primary_direction,vild_primary_name,vild_primary_destination,vild_primary_location,vild_secondary_name,vild_secondary_destination,vild_secondary_location,vild_primary_hectometer,traffic_jam_length_m,type,general_cause_type,cause_type,driving_direction,road_code
83494,2025-09-22 15:58:29,NaT,2025-09-22 16:19:08,A50,R,Sint-Oedenrode,Afrit,9474,Ekkersweijer,Knooppunt (triangle),9658.0,101.2,6100,stationaryTraffic,NonManagedCause,accident,northBound,A50
5890,2025-01-28 17:18:30,NaT,2025-01-28 17:29:13,A2,R,De Hogt,Knooppunt (triangle),7918,Batadorp,Knooppunt (triangle),7925.0,158.6,2500,slowTraffic,ManagedCause,unknown,northBound,A2


Why use tokens for location matching  

- **Location matching**  
  Used to check if a traffic jam (jam dataset) belongs to the same location as a measurement point (intensity dataset).  

- **Fuzzy match light**  
  Instead of using heavy fuzzy string matching, we break names into tokens (meaningful words).  
  This is **faster, simpler, and robust** for large datasets.  

- **Stopwords removal**  
  Words like *“naar”*, *“op”*, *“afrit”*, *“knooppunt”* are removed.  
  These do not describe a unique location and would only add noise.  

Example:  
- `"A2 northBound naar Oss op knppnt De Hogt"` → tokens = `{a2, northbound, oss, hogt}`  
- `"De Hogt"` → tokens = `{hogt}`  
- Overlap = `hogt` → **match found**

NDW (Nationale Databank Wegverkeersgegevens) documentation: [ndw.nu/datalevering](https://ndw.nu/datalevering) – explains how locations are described differently across feeds.  

In [72]:
df_files["vild_primary_name_norm"] = df_files["vild_primary_name"].map(
    normalize_ref_text)
df_files["vild_primary_tokens"] = df_files["vild_primary_name"].map(
    to_meaningful_keys)

In [86]:
df_files["end_time"] = df_files["end_time"].fillna(
    df_files["situation_version_time"]
)
df_files.sample(2)

,start_time,end_time,situation_version_time,vild_primary_road_number,vild_primary_direction,vild_primary_name,vild_primary_destination,vild_primary_location,vild_secondary_name,vild_secondary_destination,...,vild_primary_hectometer,traffic_jam_length_m,type,general_cause_type,cause_type,driving_direction,road_code,vild_primary_name_norm,vild_primary_tokens,end_time_filled
27790,2025-04-17 11:35:28,2025-04-17 11:38:10,2025-04-17 11:38:00,A2,L,Batadorp,Knooppunt (triangle),7925,Leenderheide,Knooppunt (triangle),...,164.4,2300,slowTraffic,ManagedCause,unknown,southBound,A2,batadorp,{batadorp},2025-04-17 11:38:10
62062,2025-07-04 14:15:34,2025-07-04 14:32:07,2025-07-04 14:32:07,A2,R,De Hogt,Knooppunt (triangle),7918,Batadorp/Randweg N2,Afrit,...,158.1,2400,slowTraffic,ManagedCause,unknown,northBound,A2,de hogt,{hogt},2025-07-04 14:32:07


In [87]:
df_files.isna().sum()

start_time                     0
end_time                       0
situation_version_time         0
vild_primary_road_number       0
vild_primary_direction         0
vild_primary_name              0
vild_primary_destination       0
vild_primary_location          0
vild_secondary_name           23
vild_secondary_destination    23
vild_secondary_location       23
vild_primary_hectometer       23
traffic_jam_length_m           0
type                           0
general_cause_type             0
cause_type                     0
driving_direction              0
road_code                      0
vild_primary_name_norm         0
vild_primary_tokens            0
end_time_filled                0
dtype: int64

In [ ]:
df_jam_clean = df_files.copy()

In [78]:
df_jam_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85645 entries, 0 to 85644
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   start_time                  85645 non-null  datetime64[ns]
 1   end_time                    9310 non-null   datetime64[ns]
 2   situation_version_time      85645 non-null  datetime64[ns]
 3   vild_primary_road_number    85645 non-null  object        
 4   vild_primary_direction      85645 non-null  object        
 5   vild_primary_name           85645 non-null  object        
 6   vild_primary_destination    85645 non-null  object        
 7   vild_primary_location       85645 non-null  int64         
 8   vild_secondary_name         85622 non-null  object        
 9   vild_secondary_destination  85622 non-null  object        
 10  vild_secondary_location     85622 non-null  float64       
 11  vild_primary_hectometer     85622 non-null  float64   